In [1]:
!pip install -q mediapipe==0.10.0


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!wget -O embedder.tflite -q https://storage.googleapis.com/mediapipe-models/image_embedder/mobilenet_v3_small/float32/1/mobilenet_v3_small.tflite

In [1]:
import cv2
import math
import os
import numpy as np
import pandas as pd
from tqdm import tqdm


In [2]:
def get_image_paths(directory):
    image_paths = []
    valid_extensions = ['.jpg', '.jpeg', '.png', '.gif']  # Add more extensions if needed

    for filename in os.listdir(directory):
        if any(filename.lower().endswith(ext) for ext in valid_extensions):
            image_paths.append(os.path.join(directory, filename))

    return image_paths

In [3]:
train_paths = get_image_paths("../resources/")
test_paths =  get_image_paths("../generated_images/")

In [4]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

base_options = python.BaseOptions(model_asset_path='embedder.tflite')
l2_normalize = True #@param {type:"boolean"}
quantize = True #@param {type:"boolean"}
options = vision.ImageEmbedderOptions(
    base_options=base_options, l2_normalize=l2_normalize, quantize=quantize)


embeddings = {}
test_embeddings = {}
with vision.ImageEmbedder.create_from_options(options) as embedder:
    for path in tqdm(train_paths):
        img = mp.Image.create_from_file(path)
        embeddings[path] = embedder.embed(img)
    for path in tqdm(test_paths): 
        img = mp.Image.create_from_file(path)
        test_embeddings[path] = embedder.embed(img)

2024-01-30 22:54:19.214842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-30 22:54:19.806973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
100%|██████████████████████████████████████| 831/831 [00:07<00:00, 107.38it/s]


In [5]:
embeddings

{'../resources/5_1_1_20170116153827997.png': EmbeddingResult(embeddings=[Embedding(embedding=array([  0,  13, 254, ...,   2, 255,  12], dtype=uint8), head_index=0, head_name='feature')], timestamp_ms=None),
 '../resources/5_1_3_20170119163208659.png': EmbeddingResult(embeddings=[Embedding(embedding=array([2, 0, 5, ..., 1, 2, 0], dtype=uint8), head_index=0, head_name='feature')], timestamp_ms=None),
 '../resources/5_0_2_20170116192739617.png': EmbeddingResult(embeddings=[Embedding(embedding=array([253,   9, 254, ...,  18,   2,   2], dtype=uint8), head_index=0, head_name='feature')], timestamp_ms=None),
 '../resources/6_0_1_20170117184611832.png': EmbeddingResult(embeddings=[Embedding(embedding=array([  1,   5, 254, ...,   1,   2,   0], dtype=uint8), head_index=0, head_name='feature')], timestamp_ms=None),
 '../resources/5_1_1_20170116234825295.png': EmbeddingResult(embeddings=[Embedding(embedding=array([254,  13, 254, ...,   1,   4, 254], dtype=uint8), head_index=0, head_name='feature')

In [6]:
embeddings_arr = [v.embeddings[0].embedding for k,v in embeddings.items()]
embeddings_arr

[array([  0,  13, 254, ...,   2, 255,  12], dtype=uint8),
 array([2, 0, 5, ..., 1, 2, 0], dtype=uint8),
 array([253,   9, 254, ...,  18,   2,   2], dtype=uint8),
 array([  1,   5, 254, ...,   1,   2,   0], dtype=uint8),
 array([254,  13, 254, ...,   1,   4, 254], dtype=uint8),
 array([253, 254,   7, ...,   7, 254,   6], dtype=uint8),
 array([  2, 253, 255, ...,   5,   2, 254], dtype=uint8),
 array([255,   0,   1, ...,   2,   3,   1], dtype=uint8),
 array([255,   0,  11, ...,   2,   2,   1], dtype=uint8),
 array([255, 255,   0, ...,  10,   0,   3], dtype=uint8),
 array([253, 252, 253, ..., 253,   2,   6], dtype=uint8),
 array([255,   2,   7, ...,   1, 255,   1], dtype=uint8),
 array([  0, 253,   2, ..., 254, 255,  14], dtype=uint8),
 array([253,   3,   2, ...,   2,   3, 255], dtype=uint8),
 array([253,   2, 254, ..., 254, 254,   0], dtype=uint8),
 array([254,   4,   0, ..., 255,   7,   1], dtype=uint8),
 array([254,   5,  10, ..., 255,   3,   1], dtype=uint8),
 array([254,   2,   3, ...

In [7]:
from sklearn.svm import OneClassSVM

In [29]:
nu = 0.4
kernel = "rbf" 
ocsvm = OneClassSVM(nu=nu, kernel=kernel).fit(embeddings_arr)

In [30]:
predictions = ocsvm.predict([e.embeddings[0].embedding for path, e in test_embeddings.items()])

In [31]:
predictions

array([ 1, -1,  1,  1,  1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1,  1, -1,
        1, -1,  1,  1,  1, -1, -1, -1,  1, -1, -1, -1, -1,  1,  1, -1,  1,
        1, -1,  1,  1, -1, -1, -1,  1,  1,  1, -1, -1, -1,  1,  1,  1,  1,
       -1, -1,  1, -1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,
       -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1, -1,
        1,  1,  1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1,
        1,  1,  1, -1, -1, -1,  1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,
        1,  1,  1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1, -1,
       -1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,
        1, -1, -1,  1,  1, -1,  1, -1,  1, -1, -1,  1,  1, -1, -1, -1, -1,
       -1,  1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1,  1,  1, -1,
       -1, -1, -1,  1, -1,  1, -1,  1, -1,  1,  1,  1, -1,  1,  1, -1,  1,
       -1,  1,  1, -1, -1, -1,  1, -1,  1,  1, -1, -1,  1,  1,  1, -1, -1,
        1,  1, -1,  1, -1

In [32]:
i = 0
c = 1
for path, e in test_embeddings.items():
    if predictions[i] == -1: 
        print(c, path)
        c = c + 1
    i = i + 1

1 ../generated_images/0_0_1_20230911114114779350_similar_accurate.png
2 ../generated_images/2_0_3_20231111193717106519.png
3 ../generated_images/0_0_1_20230911123959005753_random_moderate.png
4 ../generated_images/1_0_0_20230911143720372068_none_none.png
5 ../generated_images/0_1_3_20230911114631298749_similar_imprecise.png
6 ../generated_images/1_1_2_20230911115721322550_random_imprecise.png
7 ../generated_images/3_0_2_20230911144515618080_none_none.png
8 ../generated_images/1_0_0_20230911103355583053_similar_accurate.png
9 ../generated_images/5_1_0_20230911122237992872_random_moderate.png
10 ../generated_images/2_0_3_20231113141636500810.png
11 ../generated_images/2_0_3_20231113141703391736.png
12 ../generated_images/3_0_2_20231112181650398300.png
13 ../generated_images/0_1_3_20230911124527830245_random_moderate.png
14 ../generated_images/6_1_3_20230911123034558716_random_imprecise.png
15 ../generated_images/5_1_0_20230911122249550123_random_imprecise.png
16 ../generated_images/0_0_1

In [33]:
df = pd.DataFrame({
    'path': [path for path, e in test_embeddings.items()],
    'label': predictions
})

In [34]:
df

,path,label
0,../generated_images/4_1_2_20230911121712798460...,1
1,../generated_images/0_0_1_20230911114114779350...,-1
2,../generated_images/6_1_3_20230911112122099637...,1
3,../generated_images/2_1_1_20230911120526367667...,1
4,../generated_images/3_0_2_20231112181533824949...,1
...,...,...
826,../generated_images/7_1_1_20230911113547756814...,1
827,../generated_images/7_1_1_20231111200622781893...,1
828,../generated_images/1_0_0_20230911143750413289...,-1
829,../generated_images/6_0_0_20231111195832044560...,-1


In [35]:
df.to_csv(f"{nu=}_{kernel=}.csv")